In [1]:
import pandas as pd
import numpy as np

In [2]:
# kohort barn
b0, b1 = 72, 80

# kohort foreldre
f0, f1 = 40, 58

# Inntektstype, 1=samlet inntekt, 2=yrkes-og næringsinntekt, 3=pensjonsgivende inntekt 
inntekt = 1

In [3]:
df = pd.read_csv('kpi.csv', header=0, sep=';')
kpi = df[df.columns[1]]
kpi.index = df[df.columns[0]]
kpi = kpi[kpi.index <= 2015]
kpi = kpi.str.replace(',','.')
kpi = kpi.astype(float)
kpi = 100 / kpi
kpi.head()

Unnamed: 0
2015    1.000000
2014    1.021450
2013    1.042753
2012    1.064963
2011    1.071811
Name: Årsgj.snitt, dtype: float64

In [4]:
with open('test.txt', 'w') as f:
    f.write('create-dataset barn \n')
    f.write('import BEFOLKNING_FOEDSELS_AAR_MND as fodmnd \n')
    f.write('generate kohort = int( fodmnd / 100 ) \n')
    f.write('drop if kohort > 19'+str(b1)+' | kohort < 19'+str(b0) + '\n')
    f.write('import BEFOLKNING_KJOENN as mann \n')
    f.write('destring mann, force \n')
    f.write('replace mann = 0 if mann == 2 \n')
    
    f.write('import NUDB_KOMM_16 as bosted \n')
    f.write(f'destring bosted, force \n')
    f.write('drop if bosted < 100 | bosted > 2200 \n')
    f.write('drop if sysmiss(bosted) \n')

    for år in range(2,9+1):
        f.write('import BEFOLKNING_REGSTAT 200'+str(år)+'-01-01 as regstat0'+str(år)+'\n')
    for år in range(10,15+1):
        f.write('import BEFOLKNING_REGSTAT 20'+str(år)+'-01-01 as regstat'+str(år)+'\n')
    
    for kohort in range(b0, b1+1):
        string = 'drop if ('
        for år in range(kohort+31, kohort+35):
            string += f"(regstat{str(år)[1:]} != '1') | "
        string += f"(regstat{str(kohort+35)[1:]} != '1')) & kohort == 19{kohort} \n"
        f.write(string)
    f.write('tabulate kohort \n')
            
    for år in range(2002, 2015+1):
        if inntekt == 1:
            f.write('import INNTEKT_WSAMINNT_KORRIGERT '+str(år)+'-01-01 as i'+str(år)[2:]+'\n')
        elif inntekt == 2:
            f.write('import INNTEKT_WYRKINNT '+str(år)+'-01-01 as i'+str(år)[2:]+'\n')
        elif inntekt == 3:
            f.write('import INNTEKT_PGIVINNT '+str(år)+'-01-01 as i'+str(år)[2:]+'\n')
        f.write(f'replace i{str(år)[2:]} = 0 if i{str(år)[2:]} < 0 | sysmiss(i{str(år)[2:]}) \n')
        f.write('generate i'+str(år)[2:]+'ij = i'+str(år)[2:]+'*'+f'{kpi[år]:.4f}\n')

    inntekter = ['innt31', 'innt32', 'innt33','innt34','innt35']
    for inntår in inntekter:
        f.write(f'generate {inntår} = 0 \n')
    
    for kohort in range(b0, b1+1):
        f.write('replace innt31 = i'+str(kohort+31)[1:]+'ij if kohort == 19'+str(kohort)+'\n')
        f.write('replace innt32 = i'+str(kohort+32)[1:]+'ij if kohort == 19'+str(kohort)+'\n')
        f.write('replace innt33 = i'+str(kohort+33)[1:]+'ij if kohort == 19'+str(kohort)+'\n')
        f.write('replace innt34 = i'+str(kohort+34)[1:]+'ij if kohort == 19'+str(kohort)+'\n')
        f.write('replace innt35 = i'+str(kohort+35)[1:]+'ij if kohort == 19'+str(kohort)+'\n')
    f.write('generate inntekt_barn = (innt31 + innt32 + innt33 + innt34 + innt35) / 5 \n')
    f.write('tabulate kohort \n')
    f.write('import BEFOLKNING_FAR_FNR as far_fnr \n')

#####  Fedre

In [5]:
with open('test.txt', 'a+') as f:
    f.write('create-dataset far \n')
    f.write('import BEFOLKNING_FOEDSELS_AAR_MND as fodmnd \n')
    f.write('generate kohortf = int( fodmnd / 100 ) \n')
    f.write('drop if kohortf > 19'+str(f1)+'| kohortf < 19'+str(f0)+' \n')
    f.write('import BEFOLKNING_KJOENN as mann \n')
    f.write("drop if mann == '2' \n")
    for år in range(93,98+1):
        f.write('import BEFOLKNING_REGSTAT 19'+str(år)+'-01-01 as regstat'+str(år)+'\n')
    for år in range(93,98+1):
        f.write('drop if regstat'+str(år) + " != '1' \n")
        
    for år in range(1993, 1998+1):
        if inntekt == 1:
            f.write('import INNTEKT_WSAMINNT_KORRIGERT '+str(år)+'-01-01 as if'+str(år)[2:]+'\n')
        elif inntekt == 2:
            f.write('import INNTEKT_WYRKINNT '+str(år)+'-01-01 as if'+str(år)[2:]+'\n')
        elif inntekt == 3:
            f.write('import INNTEKT_PGIVINNT '+str(år)+'-01-01 as if'+str(år)[2:]+'\n') 
        f.write(f'replace if{str(år)[2:]} = 0 if (if{str(år)[2:]} < 0) | sysmiss(if{str(år)[2:]}) \n')
        f.write('generate if'+str(år)[2:]+'ij = if'+str(år)[2:]+'*'+f'{kpi[år]:.4f}\n')
    f.write('generate inntekt_far = (if93ij + if94ij + if95ij + if96ij + if97ij + if98ij) / 6 \n')
    f.write('generate prosentil_far = quantile(inntekt_far,100) \n')
    f.write('generate kvintil_far = quantile(inntekt_far,5) \n')

    f.write('merge inntekt_far prosentil_far kvintil_far kohortf into barn on far_fnr \n')

##### nasjonal

In [6]:
with open('test.txt', 'a+') as f:
    f.write('use barn \n')
    f.write('drop if sysmiss(inntekt_far) | inntekt_far <= 0 \n')
    f.write('tabulate kohort \n')
    f.write('generate alder_fodsel = kohort-kohortf \n')
    f.write('summarize alder_fodsel \n')
    f.write('tabulate kohort, summarize(alder_fodsel) \n')
    f.write('generate alder_inntekt = 1993-kohortf \n')
    f.write('tabulate kohort, summarize(alder_inntekt) \n')
    
    
    f.write('generate log_inntekt_barn = ln(inntekt_barn) \n')
    f.write('generate log_inntekt_far = ln(inntekt_far) \n')
    
    f.write('generate prosentil_barn = quantile(inntekt_barn,100) \n')
    f.write('generate kvintil_barn = quantile(inntekt_barn,5) \n')
    f.write('generate topp_kvintil = kvintil_barn==4 \n')
    f.write('tabulate kvintil_barn kvintil_far, colpct \n')
    
    f.write('tabulate prosentil_far, summarize(inntekt_far) \n')
    f.write('tabulate prosentil_far, summarize(inntekt_barn) \n')
    f.write('tabulate prosentil_far, summarize(log_inntekt_far) \n')
    f.write('tabulate prosentil_far, summarize(log_inntekt_barn) \n')
    f.write('tabulate prosentil_far, summarize(prosentil_barn) \n')
    f.write('tabulate prosentil_far if mann == 1, summarize(prosentil_barn)  \n')
    f.write('tabulate prosentil_far if mann == 0, summarize(prosentil_barn) \n')
    
    f.write('regress log_inntekt_barn log_inntekt_far \n')
    f.write('regress log_inntekt_barn log_inntekt_far if mann ==1 \n')
    f.write('regress log_inntekt_barn log_inntekt_far if mann == 0  \n')
    f.write('regress prosentil_barn prosentil_far  \n')
    f.write('regress prosentil_barn prosentil_far if mann == 1 \n')
    f.write('regress prosentil_barn prosentil_far if mann == 0 \n')
    f.write('regress inntekt_barn inntekt_far if prosentil_far <90 \n')
    f.write('regress inntekt_barn inntekt_far if prosentil_far >=90 \n')

### geografi

##### økonomiske region

In [7]:
with open('test.txt', 'a+') as f:
    f.write('replace bosted = 701 if (bosted == 717 ) | (bosted == 703 ) \n')
    f.write('replace bosted = 704 if (bosted == 721) | (bosted == 705 ) \n')
    f.write('replace bosted = 709 if (bosted == 727) | (bosted == 726 ) | (bosted == 725 ) | (bosted == 708 ) | (bosted == 707)\n')
    f.write('replace bosted = 105 if (bosted == 130) | (bosted ==  115) | (bosted == 114) | (bosted == 102) \n')
    f.write('replace bosted = 106 if (bosted == 103) | (bosted ==  113) | (bosted == 131) | (bosted == 133) | (bosted == 134) \n')
    f.write('replace bosted = 403 if (bosted == 414) | (bosted ==  401) \n')
    f.write('replace bosted = 906 if (bosted == 903) | (bosted ==  918) | (bosted == 920) | (bosted == 921) | (bosted == 922) \n')
    f.write('replace bosted = 2004 if (bosted == 2001) | (bosted ==  2016)  \n')
    f.write('replace bosted = 716 if (bosted == 718) \n')
    f.write('replace bosted = 1804 if (bosted == 1842) \n')
    f.write('replace bosted = 1576 if (bosted == 1569) | (bosted ==  1572) \n')
    f.write('replace bosted = 1160 if (bosted == 1154) | (bosted ==  1159) \n')
    f.write('replace bosted = 1505 if (bosted == 1503) | (bosted ==  1556) \n')
    f.write('replace bosted = 1756 if (bosted == 1723) | (bosted ==  1729) \n')
    f.write('replace bosted = 1903 if (bosted == 1901) | (bosted ==  1915) \n')
    f.write('replace bosted = 710 if (bosted == 706) | (bosted ==  719) | (bosted ==  720) \n')
    f.write('replace bosted = 1214 if (bosted == 1159) \n')

In [8]:
df = pd.read_csv('regioner_2017.csv',sep=";", encoding="ISO-8859-1")
# dict som mapper pendle-key til liste med kommuner
pendle_dict = {}
for key in pd.unique(df.sourceCode):
    pendle_dict[key] = list(df['targetCode'][df.sourceCode == key])
with open('test.txt', 'a+') as f:
    f.write('generate pendle = 0 \n')
    for key in pendle_dict:
        f.write(f"replace pendle = {key} if ")
        for item in pendle_dict[key][:-1]:
            f.write(f"(bosted == {item}) | ")
        f.write(f"(bosted == {pendle_dict[key][-1]}) \n")
    f.write('tabulate pendle \n')
    f.write('regress prosentil_barn c.prosentil_far##i.pendle, noconstant \n')
    f.write('regress prosentil_barn c.prosentil_far##i.pendle if mann == 1, noconstant \n')
    f.write('regress prosentil_barn c.prosentil_far##i.pendle if mann == 0, noconstant \n')

##### kommuner

In [9]:
df = pd.read_csv("kommune_n.csv",names=['kode', 'n'], header=None)
df['kode'] = [x.split('-')[0] for x in df.kode]
df = df[df.n <= 250]

string = "drop if "
for kode in df.kode[:-1]:
    string += f'(bosted == {kode}) | '
string += f'(bosted == {df.kode.iloc[-1]}) \n'

with open('test.txt','a+') as f:
    f.write(string)
    f.write('tabulate bosted \n')
    f.write('regress prosentil_barn c.prosentil_far##i.bosted, noconstant \n')
    f.write('regress topp_kvintil i.bosted if kvintil_far == 0, noconstant \n')